### LG aimers 해커톤: 1주차 데이터 전처리
- 논문을 참고하자면 논문은 IVF 체외수정"만"을 처리함. 그래서 "DI 임신 횟수"와 같은 feature들이 포함되지 않음. 제공 받은 데이터에선 DI(체내수정) 그리고 특정 시술 유형으로 또 나눠지는 것을 볼 수가 있음. 
- 우선 먼저 첫번째로 시도 하는거니까 논문대로 "IVF만" 처리를 해보고 성능이 구리다 싶음 DI혹은 특정 시술 유형을 포함하여 다시 전처리를 하는 방향으로 가야할 것 같음.
    
- 사실 DI는 냉동된, 기증된 남성의 정자를 사용하는 시술이라서 한국 정서상 필요 없을 것 같다는 생각이 들긴함.

- 성능 구릴시 차선책: DI 방식와 IVF 방식을 사용한 사람들을 나누어서 각각 모델 훈련하고 둘이 합쳐서 검증하기. trian_IVF/ train_DI

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv(r"C:\Users\DELL\OneDrive\바탕 화면\학교\대학교 3학년\LG aimers\open\train.csv", encoding="utf-8")
test = pd.read_csv(r"C:\Users\DELL\OneDrive\바탕 화면\학교\대학교 3학년\LG aimers\open\test.csv", encoding="utf-8")

print(f"train: {train.shape}\ntest: {test.shape}")


train: (256351, 69)
test: (90067, 68)


In [9]:
# DI 시술인 사람들 제외
train = train[train['시술 유형'] == "IVF"]

In [10]:
print(f"train: {train.shape}\ntest: {test.shape}")

train: (250060, 69)
test: (90067, 68)


In [6]:
train.head()

,ID,시술 시기 코드,시술 당시 나이,임신 시도 또는 마지막 임신 경과 연수,시술 유형,특정 시술 유형,배란 자극 여부,배란 유도 유형,단일 배아 이식 여부,착상 전 유전 검사 사용 여부,...,기증 배아 사용 여부,대리모 여부,PGD 시술 여부,PGS 시술 여부,난자 채취 경과일,난자 해동 경과일,난자 혼합 경과일,배아 이식 경과일,배아 해동 경과일,임신 성공 여부
0,TRAIN_000000,TRZKPL,만18-34세,NaN,IVF,ICSI,1,기록되지 않은 시행,0.0,NaN,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,3.0,NaN,0
1,TRAIN_000001,TRYBLT,만45-50세,NaN,IVF,ICSI,0,알 수 없음,0.0,NaN,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0
2,TRAIN_000002,TRVNRY,만18-34세,NaN,IVF,IVF,1,기록되지 않은 시행,0.0,NaN,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,2.0,NaN,0
3,TRAIN_000003,TRJXFG,만35-37세,NaN,IVF,ICSI,1,기록되지 않은 시행,0.0,NaN,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0
4,TRAIN_000004,TRVNRY,만18-34세,NaN,IVF,ICSI,1,기록되지 않은 시행,0.0,NaN,...,0.0,0.0,NaN,NaN,0.0,NaN,0.0,3.0,NaN,0


- 69개의 feature 중, 논문에서 사용한건 30(5개는 논문에서 어떠한 이유로 제거 => 총 25)의 feature을 사용함.

- 이게 총 30개인데 여기서 ()한 5개는 제외 한다.

[시술 당시 나이, 총 시술 횟수, IVF 임신 횟수, IVF 출산 횟수,여성 주 불임 원인,여성 부 불임 원인,  남성 주 불임 원인, 남성 부 불임 원인, 부부 주 불임 원인, 부부 부 불임 원인, 불임 원인 - 난관 질환, 불임 원인 - 배란 장애, 불임 원인 - 남성 요인, 불명확 불임 원인,  불임 원인 - 자궁내막증, 불임 원인 - 자궁경부 문제, (불임 원인 - 여성 요인), 불임 원인 - 정자 농도, 불임 원인 - 정자 형태, 불임 원인 - 정자 운동성, (불임 원인 - 정자 면역학적 요인), (배란 자극 여부), (난자 출처), (정자 출처), 신선 배아 사용 여부, 동결 배아 사용 여부, 해동된 배아 수, 수집된 신선 난자 수, 파트너 정자와 혼합된 난자 수, 이식된 배아 수]

In [ ]:
# 통계적 정보 확인
train.describe()

In [ ]:
selected_features=["시술 당시 나이", "총 시술 횟수", "IVF 임신 횟수", "IVF 출산 횟수", "여성 주 불임 원인", "여성 부 불임 원인",  "남성 주 불임 원인", "남성 부 불임 원인", "부부 주 불임 원인",
                    "부부 부 불임 원인", "불임 원인 - 난관 질환", "불임 원인 - 배란 장애", "불임 원인 - 남성 요인", "불명확 불임 원인",  "불임 원인 - 자궁내막증", "불임 원인 - 자궁경부 문제",
                    "불임 원인 - 정자 농도", "불임 원인 - 정자 형태", "불임 원인 - 정자 운동성", "신선 배아 사용 여부", "동결 배아 사용 여부", "해동된 배아 수", "수집된 신선 난자 수",
                    "파트너 정자와 혼합된 난자 수", "이식된 배아 수", "임신 성공 여부"]

# 필요한 feature 리스트 정의, "임신 성공 여부" = 타겟 변수로 70:30으로 나누어서 검증해야하기 때문에 포함시킴. 이후 x,y로 나눌 예정.
train= train[selected_features]
print(train.head())

### 데이터 전처리
- 입력데이터 (x_train, x_val)를 정규화 또는 표준화 해야됨. 그 전에..
     
 1. ~회라고 적혀 있는 문자열 -> 숫자로 변환
 2. 정규화할지 표준화 할지...
 3. 나이 묶여있는 것도 숫자로 변환
 4. 일단 DI, 특정 유형 시술 신경쓰지 말기. -> 2차 할지 말지 여부는 성능 보고
 5. 정규화/표준화 -> 정규화 선택.

- 전처리는 훈련 데이터(x_train) 기준으로 학습하여 검증 데이터(x_val)에도 동일한 변환을 적용
---
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)  # 훈련 데이터로 스케일링 학습
x_val = scaler.transform(x_val)  # 동일한 변환 적용


In [ ]:
# "시술 당시 나이" 변환
def convert_age(age):
    if age == "만18-34세":
        return 0
    elif age == "만35-37세":
        return 1
    elif age == "만38-39세":
        return 2
    elif age == "만40-42세":
        return 3
    elif age == "만43-44세":
        return 4
    elif age == "만45-50세":
        return 5
    else: # "알 수 없음"
        return None  # 범위 밖 데이터 처리

# Age 칼럼에 적용
train['시술 당시 나이'] = train['시술 당시 나이'].apply(convert_age)


In [ ]:
# 총 시술 횟수 변환
def convert_sting_to_num(count):
    if count == "0회":
        return 0
    elif count == "1회":
        return 1
    elif count == "2회":
        return 2
    elif count == "3회":
        return 3
    elif count == "4회":
        return 4
    elif count == "5회":
        return 5
    elif count == "6회 이상":
        return 6
    else:
        return None  # 알 수 없음 처리

# 총 시술 횟수 칼럼에 적용
train['총 시술 횟수'] = train['총 시술 횟수'].apply(convert_sting_to_num)

# IVF 시술 횟수 칼럼에 적용
train['IVF 임신 횟수'] = train['IVF 임신 횟수'].apply(convert_sting_to_num)

# IVF 시술 횟수 칼럼에 적용
train['IVF 출산 횟수'] = train['IVF 출산 횟수'].apply(convert_sting_to_num)

In [ ]:
# 잘 변환된 것 확인.
print(train.head())

In [ ]:
# 데이터 한국어로 좀 주지
sns.heatmap(train.corr(), annot=True, cmap='coolwarm')

In [ ]:
#결측치 수
train.isnull().sum()

# 신선/동결 배아 사용여부, 해동된 배아 수, 수집된 신선 난자 수, 파트너 정자와 혼합된 난자수, 이식된 배아 수 
# => DI 방식을 사용하였기 때문에 당연히 값이 비어있음.
# 그래서 다시 위로 올라가서 DI 방식한 사람들 싹 제거해줄 거임. 철저히 IVF만 보겠다.

In [ ]:
# 여기서 정규화 해야됨.

In [ ]:
# 입력 데이터(x)와 타겟 데이터(y) 분리
x = train.drop(columns=["임신 성공 여부"])
y = train["임신 성공 여부"]

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(
    x, y, test_size=0.3, random_state=42, stratify=y  # 0.3 = 30%를 검증에 이용하겠다.
)
# x_train: 훈련용 입력값 데이터, outcome 제외한 feature
# x_val: 검증용 입력값 데이터, outcome 제외한 feature
# y_train: 훈련용 타겟값=답(임신 성공 여부)
# y_val: 검증용 타겟값

In [ ]:
print("훈련 데이터 크기:", x_train.shape, y_train.shape)
print("검증 데이터 크기:", x_val.shape, y_val.shape)